## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Khatanga,RU,71.9667,102.5000,5.02,83,7,9.89,clear sky
1,1,Busselton,AU,-33.6500,115.3333,72.12,45,0,12.21,clear sky
2,2,Airai,TL,-8.9266,125.4092,67.86,81,73,1.95,broken clouds
3,3,Jamestown,US,42.0970,-79.2353,38.17,65,20,6.91,few clouds
4,4,Vaini,TO,-21.2000,-175.2000,73.56,94,20,4.61,few clouds


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 100


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()

In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                296
City                   296
Country                296
Lat                    296
Lng                    296
Max Temp               296
Humidity               296
Cloudiness             296
Wind Speed             296
Weather Description    296
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Busselton,AU,72.12,clear sky,-33.6500,115.3333,
4,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,
5,Georgetown,MY,85.95,few clouds,5.4112,100.3354,
7,Nautla,MX,75.09,clear sky,20.2167,-96.7833,
11,Caravelas,BR,74.57,few clouds,-17.7125,-39.2481,
14,Maceio,BR,72.84,scattered clouds,-9.6658,-35.7353,
18,Makanya,TZ,84.27,scattered clouds,-4.3667,37.8333,
21,Saint-Philippe,RE,83.79,light rain,-21.3585,55.7679,
22,Ampanihy,MG,92.16,clear sky,-24.7000,44.7500,
24,Faanui,PF,82.69,overcast clouds,-16.4833,-151.7500,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Busselton,AU,72.12,clear sky,-33.6500,115.3333,Observatory Guest House
4,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Georgetown,MY,85.95,few clouds,5.4112,100.3354,Cititel Penang
7,Nautla,MX,75.09,clear sky,20.2167,-96.7833,Casa Blanca
11,Caravelas,BR,74.57,few clouds,-17.7125,-39.2481,Pousada dos Navegantes


In [32]:
# 7. Drop the rows where there is no Hotel Name.b
# hotel_df = hotel_df.loc[(hotel_df["Hotel Name"])].dropna()

In [34]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and </dd><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [43]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Take a screenshot of your map and save it to the Vacation_Search folder as WeatherPy_vacation_map.png